In [ ]:
from confluent_kafka import Consumer

conf = {"bootstrap.servers": "localhost: 9092", "group.id": "mygroup", "auto.offset.reset": "earliest"}
consumer = Consumer(conf)
consumer.subscribe(["payment_msg"])
rows = []
for msg in consumer.consume(100):
    rows.append(msg.value().decode("utf-8"))

## Local execution on DuckDB

In [ ]:
import json

import pandas as pd

df = pd.DataFrame([json.loads(row) for row in rows])
df["createTime"] = pd.to_datetime(df["createTime"])
df

In [ ]:
import ibis

con = ibis.get_backend()
t = con.create_table("payments", df)
t

In [ ]:
t.schema()

### Example transformations: filter, transform, aggregations

In [ ]:
from ibis import _

t.filter(_.payAmount > 10000)

In [ ]:
t.filter(_.payAmount > 10000).execute()

### Transformation chaining

In [ ]:
provinces = (
    "Beijing",
    "Shanghai",
    "Hangzhou",
    "Shenzhen",
    "Jiangxi",
    "Chongqing",
    "Xizang",
)
province_id_to_name_df = pd.DataFrame(
    enumerate(provinces), columns=["provinceId", "province"]
)
province_id_to_name_df

In [ ]:
join_filter = t.join(province_id_to_name_df, ["provinceId"]).filter(
    _.payAmount > 10000
)
join_filter.execute()

In [ ]:
join_agg = t.join(province_id_to_name_df, ["provinceId"]).filter(
    _.payAmount > 10000
).group_by(["payPlatform", "provinceId"]).aggregate(
    totalPayAmount=_.payAmount.sum()
).order_by(
    _.totalPayAmount.desc()
)

In [ ]:
join_agg.visualize()

In [ ]:
print(join_agg.compile())

In [ ]:
join_agg.execute()

### Intersperse SQL with Ibis methods

In [ ]:
t.sql("SELECT * FROM payments WHERE payAmount > 10000").join(province_id_to_name_df, ["provinceId"]).execute()

### Window aggregation

In [ ]:
window_agg = t[
    "provinceId",
    _.payAmount.sum()
    .over(range=(-ibis.interval(seconds=10), 0), order_by=_.createTime)
    .name("pay_amount"),
]
window_agg

In [ ]:
window_agg.execute()

## Execute on Spark

In [ ]:
from pyspark.sql import SparkSession

session = SparkSession.builder \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1")\
    .config("spark.sql.streaming.schemaInference", True)\
    .config("spark.ui.port","4050")\
    .getOrCreate()

In [ ]:
schema = ibis.schema(
    {
        "createTime": "timestamp(3)",
        "orderId": "int64",
        "payAmount": "float64",
        "payPlatform": "int32",
        "provinceId": "int32",
    }
)

con = ibis.pyspark.connect(session, mode="streaming")

### Connect to a streaming source

In [ ]:
con.read_kafka(
    "payments", 
    watermark=ibis.watermark(time_col="createTime", allowed_delay=ibis.interval(seconds=10)), 
    schema=schema,
    auto_parse=True,
    options={"kafka.bootstrap.servers": "localhost:9092", "subscribe": "payment_msg", "startingOffsets": "earliest"}
)

In [ ]:
t = con.table("payments")
join_filter = t.join(province_id_to_name_df, ["provinceId"]).filter(
    _.payAmount > 10000
)

### Write results to sink

In [ ]:
path = "output"
con.to_csv_dir(
    join_filter,
    path=path,
    options={"checkpointLocation": "checkpoint", "header": True},
)

In [ ]:
from pathlib import Path

path = Path("output")
output = pd.concat([pd.read_csv(f) for f in path.glob("*.csv")])
output

In [ ]:
con.to_kafka(
    join_filter, 
    auto_format=True, 
    options={"checkpointLocation": "checkpoint", "kafka.bootstrap.servers": "localhost:9092", "topic": "sink"},
)